In [ ]:
import numpy as np
import graphinglib as gl
from collections import namedtuple
import astropy.units as u

from src.hdu.cubes.cube_co import CubeCO
from src.spectrums.spectrum_co import SpectrumCO
from src.spectrums.spectrum import Spectrum
from src.hdu.tesseract import Tesseract
from src.hdu.maps.grouped_maps import GroupedMaps
from src.hdu.maps.convenient_funcs import get_FWHM, get_speed
from src.coordinates.ds9_coords import DS9Coords

# Loop4N1

In [ ]:
N1 = CubeCO.load("data/Loop4_co/N1/13co/Loop4N1_13co.fits")[500:800,:,:]
N1.header["COMMENT"] = "Loop4N1_13co was sliced at channel 500; all values of mean must then be " \
                     + "added to 500 to account for this shift."

## Polyfit adjustments

In [ ]:
correction = np.full_like(N1.data, 0)
for y, map_ in enumerate(N1):
    for x, spectrum in enumerate(map_):
        if x <= 19 and y >= 31:
            if not spectrum.isnan:
                polyfit = spectrum.polyfit(3)
                correction[:,y,x] = polyfit.function(np.arange(300))
                # print(x+1, y+1)
                # fig = gl.Figure(size=(10,7), figure_style="dark")
                # fig.add_elements(spectrum.plot, polyfit)
                # fig.show()

N1.data -= correction
N1.header["COMMENT"] = "Loop4N1_13co was corrected for incorrect polyfit adjustment for upper left pixels."
N1.save("data/Loop4_co/N1/13co/Loop4N1_13co_corrected.fits")

## Fitting

In [ ]:
N1 = CubeCO.load("data/Loop4_co/N1/13co/Loop4N1_13co_corrected.fits")

if __name__ == "__main__":
    spectrum_parameters = {
        "PEAK_PROMINENCE" : 0.2,
        "PEAK_MINIMUM_HEIGHT_SIGMAS" : 4,
        "PEAK_MINIMUM_DISTANCE" : 6,
        "PEAK_WIDTH" : 3,
        "NOISE_CHANNELS" : slice(0,150),
        "INITIAL_GUESSES_BINNING" : 2,
        "MAX_RESIDUE_SIGMAS" : 100,
        "STDDEV_DETECTION_THRESHOLD" : 0.1,
        "INITIAL_GUESSES_MAXIMUM_GAUSSIAN_STDDEV" : 10,
        "INITIAL_GUESSES_MINIMUM_GAUSSIAN_STDDEV" : 1,
    }

    N1.header["COMMENT"] = f"FITTING PARAMETERS "
    for key, value in spectrum_parameters.items():
        N1.header["COMMENT"] = f"{key} = {value}"

    chi2, fit_results = N1.fit(spectrum_parameters)
    chi2.save("data/Loop4_co/N1/13co/chi2.fits")
    fit_results.save("data/Loop4_co/N1/13co/tesseract.fits")

## Verifications

In [ ]:
tesseract_N1 = Tesseract.load(f"data/Loop4_co/N1/13co/tesseract.fits")
fig = gl.Figure(size=(10,7), figure_style="dim")
fig.add_elements(*tesseract_N1.get_spectrum_plot(N1, DS9Coords(10, 37)))
fig.show()

# Loop4N2

In [ ]:
N2 = CubeCO.load("data/Loop4_co/N2/13co/Loop4N2_13co.fits")[3200:4000,:,:]
N2.header["COMMENT"] = "Loop4N2_13co was sliced at channel 3200; all values of mean must then be " \
                     + "added to 3200 to account for this shift."

## Fitting

In [ ]:
if __name__ == "__main__":
    spectrum_parameters = {
        "PEAK_PROMINENCE" : 0.2,
        "PEAK_MINIMUM_HEIGHT_SIGMAS" : 4,
        "PEAK_MINIMUM_DISTANCE" : 200,  # Force a single component
        "PEAK_WIDTH" : 3,
        "NOISE_CHANNELS" : slice(0,600),
        "INITIAL_GUESSES_BINNING" : 2,
        "MAX_RESIDUE_SIGMAS" : 100,
        "STDDEV_DETECTION_THRESHOLD" : 0.1,
        "INITIAL_GUESSES_MAXIMUM_GAUSSIAN_STDDEV" : 10,
        "INITIAL_GUESSES_MINIMUM_GAUSSIAN_STDDEV" : 1,
    }

    N2.header["COMMENT"] = f"FITTING PARAMETERS "
    for key, value in spectrum_parameters.items():
        N2.header["COMMENT"] = f"{key} = {value}"

    chi2, fit_results = N2.fit(spectrum_parameters)
    chi2.save("data/Loop4_co/N2/13co/chi2.fits")
    fit_results.save("data/Loop4_co/N2/13co/tesseract.fits")

## Verifications

In [ ]:
tesseract_N2 = Tesseract.load(f"data/Loop4_co/N2/13co/tesseract.fits")
fig = gl.Figure(size=(10,7), figure_style="dim")
fig.add_elements(*tesseract_N2.get_spectrum_plot(N2, DS9Coords(5, 10)))
fig.show()

# Loop4N4

Warning : the N4S4 cube needs to be fitted on its own as it does not feature the same spectral resolution as the other cubes.

In [ ]:
N4 = CubeCO.load("data/Loop4_co/N4/13co/Loop4N4_13co.fits")[3200:4000,:,:]
N4.header["COMMENT"] = "Loop4N4_13co was sliced at channel 3200; all values of mean must then be " \
                     + "added to 3200 to account for this shift."

## Fitting

In [ ]:
if __name__ == "__main__":
    spectrum_parameters = {
        "PEAK_PROMINENCE" : 0.2,
        "PEAK_MINIMUM_HEIGHT_SIGMAS" : 4,
        "PEAK_MINIMUM_DISTANCE" : 200,  # Force a single component
        "PEAK_WIDTH" : 3,
        "NOISE_CHANNELS" : slice(0,600),
        "INITIAL_GUESSES_BINNING" : 2,
        "MAX_RESIDUE_SIGMAS" : 100,
        "STDDEV_DETECTION_THRESHOLD" : 0.1,
        "INITIAL_GUESSES_MAXIMUM_GAUSSIAN_STDDEV" : 10,
        "INITIAL_GUESSES_MINIMUM_GAUSSIAN_STDDEV" : 1,
    }

    N4.header["COMMENT"] = f"FITTING PARAMETERS "
    for key, value in spectrum_parameters.items():
        N4.header["COMMENT"] = f"{key} = {value}"

    chi2, fit_results = N4.fit(spectrum_parameters)
    chi2.save("data/Loop4_co/N4/13co/chi2.fits")
    fit_results.save("data/Loop4_co/N4/13co/tesseract.fits")

## Verifications

In [ ]:
tesseract_N4 = Tesseract.load(f"data/Loop4_co/N4/13co/tesseract.fits")
fig = gl.Figure(size=(10,7), figure_style="dim")
fig.add_elements(*tesseract_N4.get_spectrum_plot(N4, DS9Coords(5, 10)))
fig.show()

## N4S4 cube

In [ ]:
N4S4 = CubeCO.load("data/Loop4_co/N4/13co/N4S4.fits")[400:750,:,:]
N4S4.header["COMMENT"] = "N4S4 was sliced at channel 400; all values of mean must then be " \
                     + "added to 400 to account for this shift."

if __name__ == "__main__":
    spectrum_parameters = {
        "PEAK_PROMINENCE" : 0.2,
        "PEAK_MINIMUM_HEIGHT_SIGMAS" : 4,
        "PEAK_MINIMUM_DISTANCE" : 200,  # Force a single component
        "PEAK_WIDTH" : 3,
        "NOISE_CHANNELS" : slice(0,200),
        "INITIAL_GUESSES_BINNING" : 2,
        "MAX_RESIDUE_SIGMAS" : 100,
        "STDDEV_DETECTION_THRESHOLD" : 0.1,
        "INITIAL_GUESSES_MAXIMUM_GAUSSIAN_STDDEV" : 10,
        "INITIAL_GUESSES_MINIMUM_GAUSSIAN_STDDEV" : 1,
    }

    N4S4.header["COMMENT"] = f"FITTING PARAMETERS "
    for key, value in spectrum_parameters.items():
        N4S4.header["COMMENT"] = f"{key} = {value}"

    chi2, fit_results = N4S4.fit(spectrum_parameters)
    chi2.save("data/Loop4_co/N4/13co/chi2_N4S4.fits")
    fit_results.save("data/Loop4_co/N4/13co/tesseract_N4S4.fits")

### Verifications

In [ ]:
tesseract_N4S4 = Tesseract.load(f"data/Loop4_co/N4/13co/tesseract_N4S4.fits")
fig = gl.Figure(size=(10,7), figure_style="dim")
fig.add_elements(*tesseract_N4S4.get_spectrum_plot(N4S4, DS9Coords(3, 3)))
fig.show()

### Assembling the tesseracts
The N4S4 subregion has been chosen to not be included in the final Tesseract as it only presents two pixels with fitted gaussians. The complications of having a Tesseract that comes from multiple spectral resolutions when it comes to calculations such as standard deviation greatly outweighs the very small quantity of relevant data of this subcube.

# Loop4p

In [ ]:
p = CubeCO.load("data/Loop4_co/p/13co/Loop4p_13co.fits")[400:800,:,:]
p.header["COMMENT"] = "Loop4p_13co was sliced at channel 400; all values of mean must then be " \
                    + "added to 400 to account for this shift."

## Fitting

In [ ]:
if __name__ == "__main__":
    spectrum_parameters = {
        "PEAK_PROMINENCE" : 0.1,
        "PEAK_MINIMUM_HEIGHT_SIGMAS" : 4,
        "PEAK_MINIMUM_DISTANCE" : 4,
        "PEAK_WIDTH" : 2,
        "NOISE_CHANNELS" : slice(0,200),
        "INITIAL_GUESSES_BINNING" : 1,
        "MAX_RESIDUE_SIGMAS" : 4,
        "STDDEV_DETECTION_THRESHOLD" : 0.1,
        "INITIAL_GUESSES_MAXIMUM_GAUSSIAN_STDDEV" : 10,
        "INITIAL_GUESSES_MINIMUM_GAUSSIAN_STDDEV" : 1,
    }
    
    p.header["COMMENT"] = f"FITTING PARAMETERS "
    for key, value in spectrum_parameters.items():
        p.header["COMMENT"] = f"{key} = {value}"

    chi2, fit_results = p.fit(spectrum_parameters)
    chi2.save("data/Loop4_co/p/13co/chi2.fits")
    fit_results.save("data/Loop4_co/p/13co/tesseract.fits")

## Verifications

In [ ]:
tesseract_p = Tesseract.load(f"data/Loop4_co/p/13co/tesseract.fits")
p = CubeCO.load("data/Loop4_co/p/13co/Loop4p_13co.fits")[400:800,:,:]

# for i in range(0, p.data.size // 400):
#     y, x = np.unravel_index(i, p.data.shape[1:])
#     fig = gl.Figure(size=(10,7), figure_style="dim")
#     try:
#         fig.add_elements(*tesseract_p.get_spectrum_plot(p, (int(y), int(x))))
#         print(x+1, y+1)
#         fig.show()
#     except KeyError:
#         continue
    
tesseract_p = Tesseract.load(f"data/Loop4_co/p/13co/tesseract.fits")
fig = gl.Figure(size=(10,7), figure_style="dim")
fig.add_elements(*tesseract_p.get_spectrum_plot(p, DS9Coords(24, 19)))
fig.show()


## Refits

In [ ]:
tesseract_p = Tesseract.load(f"data/Loop4_co/p/13co/tesseract.fits")

spectrum_parameters = {
    "PEAK_PROMINENCE" : 0.1,
    "PEAK_MINIMUM_HEIGHT_SIGMAS" : 4,
    "PEAK_MINIMUM_DISTANCE" : 4,
    "PEAK_WIDTH" : 2,
    "NOISE_CHANNELS" : slice(0,200),
    "INITIAL_GUESSES_BINNING" : 1,
    "MAX_RESIDUE_SIGMAS" : 4,
    "STDDEV_DETECTION_THRESHOLD" : 0.1,
    "INITIAL_GUESSES_MAXIMUM_GAUSSIAN_STDDEV" : 10,
    "INITIAL_GUESSES_MINIMUM_GAUSSIAN_STDDEV" : 1,
}

# Working in DS9Coods
spectrums = {
    (26, 13) : p[:,*DS9Coords(26, 13)],
    (25, 15) : p[:,*DS9Coords(25, 15)],
    (28, 28) : p[:,*DS9Coords(28, 28)],
    (29, 29) : p[:,*DS9Coords(29, 29)],
    (22, 30) : p[:,*DS9Coords(22, 30)],
    (21, 32) : p[:,*DS9Coords(21, 32)],
    (15, 41) : p[:,*DS9Coords(15, 41)],
    (15, 40) : p[:,*DS9Coords(15, 40)],
}
special_spectrums = {
    (25, 16) : p[:,*DS9Coords(25, 16)],
}

[spectrum.setattrs(spectrum_parameters) for spectrum in (spectrums | special_spectrums).values()]
spectrums[(25, 15)].setattrs({"PEAK_MINIMUM_HEIGHT_SIGMAS": 2.75})
spectrums[(28, 28)].setattrs({"PEAK_MINIMUM_HEIGHT_SIGMAS": 3.75})
spectrums[(22, 30)].setattrs({"PEAK_MINIMUM_HEIGHT_SIGMAS": 3.75})
spectrums[(21, 32)].setattrs({"PEAK_MINIMUM_HEIGHT_SIGMAS": 3.5})

[spectrum.fit() for spectrum in spectrums.values()]

special_spectrums[(25, 16)].fit({"stddev" : (0,5)*u.um})

for coords, spectrum in (spectrums | special_spectrums).items():
    fig = gl.Figure(figure_style="dark", size=(10, 7), title=coords)
    fig.add_elements(spectrum.plot, spectrum.initial_guesses_plot, *spectrum.individual_functions_plot,
                     spectrum.total_functions_plot)
    fig.show()

# Loop on all the fit results, adapt the removed component(s) to every spectra
tesseract_p[0,*DS9Coords(26, 13)] = np.NAN
for i, fit_result in enumerate(spectrums[(25, 15)].fit_results.to_numpy()):
    tesseract_p[i,*DS9Coords(25, 15)] = fit_result
for i, fit_result in enumerate(spectrums[(28, 28)].fit_results.to_numpy()):
    tesseract_p[i,*DS9Coords(28, 28)] = fit_result
tesseract_p[1,*DS9Coords(29, 29)] = np.NAN
for i, fit_result in enumerate(spectrums[(22, 30)].fit_results.to_numpy()):
    tesseract_p[i,*DS9Coords(22, 30)] = fit_result
tesseract_p[0,*DS9Coords(21, 32)] = np.NAN
tesseract_p[0,*DS9Coords(15, 41)] = np.NAN
tesseract_p[1,*DS9Coords(15, 40)] = np.NAN
for i, fit_result in enumerate(special_spectrums[(25, 16)].fit_results.to_numpy()):
    tesseract_p[i,*DS9Coords(25, 16)] = fit_result
tesseract_p[2,*DS9Coords(23, 17)] = np.NAN

tesseract_p.compress().save("data/Loop4_co/p/13co/object_filtered.fits")